In [1]:
import polars as pl

from src.lib import version_extractor, name_extractor

In [2]:
# pl.cfg.Config.set_tbl_rows(100)
df = pl.read_json('../../data/big_query/bq_results.json', json_lines=True)

In [3]:
df_sorted = df.sort(['name', 'version', 'upload_time'], reverse=[False, True, True]).clone()
print(df_sorted)

shape: (6777547, 4)
┌───────────────┬─────────┬─────────────────────────────────────┬────────────────────────────────┐
│ name          ┆ version ┆ requires_dist                       ┆ upload_time                    │
│ ---           ┆ ---     ┆ ---                                 ┆ ---                            │
│ str           ┆ str     ┆ list[str]                           ┆ str                            │
╞═══════════════╪═════════╪═════════════════════════════════════╪════════════════════════════════╡
│ 0             ┆ 0.0.0   ┆ []                                  ┆ 2017-08-06 03:43:27.195869 UTC │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0-core-client ┆ 1.1.0a8 ┆ ["redis (>=2.10.5)"]                ┆ 2017-10-11 13:19:49.81603 UTC  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0-core-client ┆ 1.1.0a8 ┆ ["redis (>=2.10.5)"]                ┆ 2017-10-11 13:19:47.668

In [7]:
# Eliminate duplicates
df_unique = df_sorted.unique(subset=['name', 'version'])
df_unique.head(10).to_pandas().to_json(orient='records')
# print(df_unique)
# df_unique.to_parquet('../../data/big_query/bq_results_unique.parquet')

'[{"name":"0","version":"0.0.0","requires_dist":[],"upload_time":"2017-08-06 03:43:27.195869 UTC"},{"name":"0-core-client","version":"1.1.0a8","requires_dist":["redis (>=2.10.5)"],"upload_time":"2017-10-11 13:19:49.81603 UTC"},{"name":"0-core-client","version":"1.1.0a7","requires_dist":["redis (>=2.10.5)"],"upload_time":"2017-08-08 18:10:31.426796 UTC"},{"name":"0-core-client","version":"1.1.0a5","requires_dist":["redis (>=2.10.5)"],"upload_time":"2017-07-17 06:41:55.349782 UTC"},{"name":"0-core-client","version":"1.1.0a4","requires_dist":[],"upload_time":"2017-06-21 15:00:03.092239 UTC"},{"name":"0-core-client","version":"1.1.0a3","requires_dist":["redis (>=2.10.5)"],"upload_time":"2017-06-09 08:34:43.85207 UTC"},{"name":"0-orchestrator","version":"1.1.0a8","requires_dist":["Jinja2","aioredis","etcd3","netaddr","python-dateutil"],"upload_time":"2017-10-11 13:21:50.027615 UTC"},{"name":"0-orchestrator","version":"1.1.0a7","requires_dist":["Jinja2","python-dateutil"],"upload_time":"2017

In [5]:
# def convert_to_normalized_format(row: tuple):
#     name, version, requires_dist, upload_time = row
#     result = {
#         'name': name,
#         'versions': {
#             version: {
#                 'timestamp': upload_time,
#                 'dependencies': {}
#             }
#         }
#     }
#     for semantic_versioning in requires_dist:
#         dependency_name = name_extractor(semantic_versioning)
#         dependency_version = version_extractor(semantic_versioning)
#         result['versions'][version]['dependencies'] |= {dependency_name: dependency_version}
#     return 1

def convert_to_normalized_format(struct: dict):
    name = struct['name']
    version: str = struct['version']
    requires_dist = struct['requires_dist']
    upload_time = struct['upload_time']
    result = {
        'name': name,
        'versions': {
            version: {
                'timestamp': upload_time,
                'dependencies': {}
            }
        }
    }
    for semantic_versioning in requires_dist:
        dependency_name = name_extractor(semantic_versioning)
        dependency_version = version_extractor(semantic_versioning)
        result['versions'][version]['dependencies'] |= {dependency_name: dependency_version}
    # print(result)
    return 1

# hello = df_unique.head(10).select([
#     pl.struct(['name', 'version', 'requires_dist', 'upload_time']).apply(convert_to_normalized_format)
# ])
# print(hello)
# h = df_unique.head(10).apply(convert_to_normalized_format)
# print(h)

In [6]:
# h = df_unique.head(10).apply(convert_to_normalized_format)
# print(h)

In [7]:
from typing import List


def extract_name_and_version_from_list(dependencies: List[str]) -> [(str, str)]:
    print(dependencies)
    results: List[(str, str)] = []
    for el in dependencies:
        name = name_extractor(el)
        version = version_extractor(el)
        if name and version:
            results.append((name, version))

    return results

In [8]:
# hello = pl.col("").map(extract_name_and_version_from_list)
# df_unique['requires_dist'].head().arr.eval(hello)
# df_unique.apply(convert_to_normalized_format)
# df_unique.select([
#
# ])
# df_unique.head().ma
# df_unique.select()

exploded = df_unique.explode("requires_dist").clone()
exploded

name,version,requires_dist,upload_time
str,str,str,str
"""0-core-client""","""1.1.0a8""","""redis (>=2.10.5)""","""2017-10-11 13:19:49.81603 UTC"""
"""0-core-client""","""1.1.0a7""","""redis (>=2.10.5)""","""2017-08-08 18:10:31.426796 UTC"""
"""0-core-client""","""1.1.0a5""","""redis (>=2.10.5)""","""2017-07-17 06:41:55.349782 UTC"""
"""0-core-client""","""1.1.0a3""","""redis (>=2.10.5)""","""2017-06-09 08:34:43.85207 UTC"""
"""0-orchestrator""","""1.1.0a8""","""Jinja2""","""2017-10-11 13:21:50.027615 UTC"""
"""0-orchestrator""","""1.1.0a8""","""aioredis""","""2017-10-11 13:21:50.027615 UTC"""
"""0-orchestrator""","""1.1.0a8""","""etcd3""","""2017-10-11 13:21:50.027615 UTC"""
"""0-orchestrator""","""1.1.0a8""","""netaddr""","""2017-10-11 13:21:50.027615 UTC"""
"""0-orchestrator""","""1.1.0a8""","""python-dateutil""","""2017-10-11 13:21:50.027615 UTC"""


In [9]:
dependency_version_series = exploded['requires_dist'].apply(version_extractor)
dependency_name_series = exploded['requires_dist'].apply(name_extractor)

In [10]:
exploded.drop_in_place('requires_dist')
exploded['dependency_name'] = dependency_name_series
exploded['dependency_version'] = dependency_version_series
exploded

name,version,upload_time,dependency_name,dependency_version
str,str,str,str,str
"""0-core-client""","""1.1.0a8""","""2017-10-11 13:19:49.81603 UTC""","""redis""",""">=2.10.5"""
"""0-core-client""","""1.1.0a7""","""2017-08-08 18:10:31.426796 UTC""","""redis""",""">=2.10.5"""
"""0-core-client""","""1.1.0a5""","""2017-07-17 06:41:55.349782 UTC""","""redis""",""">=2.10.5"""
"""0-core-client""","""1.1.0a3""","""2017-06-09 08:34:43.85207 UTC""","""redis""",""">=2.10.5"""
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","""Jinja2""",""">=0.0.0"""
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","""aioredis""",""">=0.0.0"""
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","""etcd3""",""">=0.0.0"""
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","""netaddr""",""">=0.0.0"""
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","""python-dateutil""",""">=0.0.0"""


In [11]:
print(len(exploded.groupby('name').groups()))

116773


In [ ]:
from multiprocessing.pool import Pool
import multiprocessing
from polars import DataFrame

def convert_to_normalized_format(grouped_df: DataFrame):
    print(grouped_df)
    normalized_form = {
        # We know the name is the same for all rows
        'name': grouped_df['name'][0],
        'versions': {}
    }
    for index, version in enumerate(grouped_df['version']):
        normalized_form['versions'][version] = {
            'timestamp': grouped_df['upload_time'][0],
            'dependencies': {}
        }
        for dependency, dependency_version in zip(grouped_df['dependency_name'], grouped_df['dependency_version']):
            normalized_form['versions'][version]['dependencies'][dependency] = dependency_version

    # global_dict |= normalized_form
    # Does not work since apply expects a data frame as a return
    return normalized_form

cores = multiprocessing.cpu_count()
# chunks = np.array_split(info_df['dependency'], cores)
small = exploded.head(10)
with Pool(cores) as pool:
    results = pool.map(convert_to_normalized_format, small.groupby('name'))

results
#     processed = pd.concat(pool.map(extract_semantic_version, chunks), ignore_index=True)
# results = list(map(convert_to_normalized_format, exploded.groupby('name')))
# exploded.select([
#     pl.struct(['name', 'version', 'upload_time', 'dependency_name', 'dependency_version']).apply(convert_to_normalized_format)
# ])

In [12]:
results

[{'name': 'ArgDoc',
  'versions': {'0.1.3b2': {'timestamp': '2018-12-05 21:53:52.184072 UTC',
    'dependencies': {'future': '>=0.0.0',
     'sphinx': '>=0.0.0',
     'sphinxcontrib-programoutput': '>=0.0.0'}},
   '0.1.3b1': {'timestamp': '2018-12-05 21:53:52.184072 UTC',
    'dependencies': {'future': '>=0.0.0',
     'sphinx': '>=0.0.0',
     'sphinxcontrib-programoutput': '>=0.0.0'}},
   '0.1.2b1': {'timestamp': '2018-12-05 21:53:52.184072 UTC',
    'dependencies': {'future': '>=0.0.0',
     'sphinx': '>=0.0.0',
     'sphinxcontrib-programoutput': '>=0.0.0'}},
   '0.1.2': {'timestamp': '2018-12-05 21:53:52.184072 UTC',
    'dependencies': {'future': '>=0.0.0',
     'sphinx': '>=0.0.0',
     'sphinxcontrib-programoutput': '>=0.0.0'}},
   '0.1.1': {'timestamp': '2018-12-05 21:53:52.184072 UTC',
    'dependencies': {'future': '>=0.0.0',
     'sphinx': '>=0.0.0',
     'sphinxcontrib-programoutput': '>=0.0.0'}}}},
 {'name': 'ARC-Alkali-Rydberg-Calculator',
  'versions': {'3.2.0': {'timest

In [14]:
import json

with open('../../data/output/bq_results.json', 'w') as file:
    json.dump(results, file)

In [10]:
exploded.to_parquet('../../data/big_query/bq_results_unique_exploded.parquet')

In [14]:
# versions = exploded['requires_dist'].str.replace(r'\)', '(').str.split('(')
# versions
separated_name_and_version = exploded.select([
    pl.all().exclude('requires_dist'),
    pl.col('requires_dist').str.replace(r'\)', '(').str.split('(').arr.eval(pl.col('').str.strip())
])
separated_name_and_version
# exploded.head().with_columns([
#     pl.col('requires_dist').arr.first()
# ])

name,version,upload_time,requires_dist
str,str,str,list[str]
"""0-core-client""","""1.1.0a8""","""2017-10-11 13:19:49.81603 UTC""","[""redis"", "">=2.10.5"", """"]"
"""0-core-client""","""1.1.0a7""","""2017-08-08 18:10:31.426796 UTC""","[""redis"", "">=2.10.5"", """"]"
"""0-core-client""","""1.1.0a5""","""2017-07-17 06:41:55.349782 UTC""","[""redis"", "">=2.10.5"", """"]"
"""0-core-client""","""1.1.0a3""","""2017-06-09 08:34:43.85207 UTC""","[""redis"", "">=2.10.5"", """"]"
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","[""Jinja2""]"
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","[""aioredis""]"
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","[""etcd3""]"
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","[""netaddr""]"
"""0-orchestrator""","""1.1.0a8""","""2017-10-11 13:21:50.027615 UTC""","[""python-dateutil""]"


In [17]:
# Remember: think about everything as columns operations
hello = separated_name_and_version.with_columns([
    pl.col('requires_dist').arr.get(0).alias('dependency_name'),
    # pl.when(pl.col('requires_dist').arr.lengths() > 1).then(pl.col('requires_dist').arr.get(1)).otherwise('*').alias('dependency_version')
    pl.when(pl.col('requires_dist').arr.lengths() > 1).then(pl.col('requires_dist').arr.get(1)).otherwise('*').alias('dependency_version')
])
print(hello)
# separated_name_and_version['requires_dist'].arr.get(0)

shape: (7338985, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ name           ┆ version        ┆ upload_time    ┆ requires_dist ┆ dependency_na ┆ dependency_ve │
│ ---            ┆ ---            ┆ ---            ┆ ---           ┆ me            ┆ rsion         │
│ str            ┆ str            ┆ str            ┆ list[str]     ┆ ---           ┆ ---           │
│                ┆                ┆                ┆               ┆ str           ┆ str           │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 0-core-client  ┆ 1.1.0a8        ┆ 2017-10-11     ┆ ["redis",     ┆ redis         ┆ >=2.10.5      │
│                ┆                ┆ 13:19:49.81603 ┆ ">=2.10.5",   ┆               ┆               │
│                ┆                ┆ UTC            ┆ ""]           ┆               ┆               │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌